In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import seaborn as sns
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import keras as ks
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau




2025-01-06 13:20:20.637906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
db_path='./triple_mnist/'

def flatten_toarray(set):
    set_path=db_path+set
    sub_folders=os.listdir(set_path)

    imgs=[]
    labels=[]

    for folder in sub_folders:
        subfolder_path = os.path.join(set_path, folder)

        for img_file in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, img_file)

            flattened_img = (Image.open(img_path))
            flattened_img = (np.array((flattened_img).convert('L')))

            imgs.append(flattened_img)
            labels.append(int(folder))
            
    return np.array(imgs), np.array(labels)


f_train_imgs, f_train_labels = flatten_toarray('train')
f_test_imgs, f_test_labels = flatten_toarray('test')
f_val_imgs, f_val_labels = flatten_toarray('val')

f_train_imgs = f_train_imgs / 255.0
f_test_imgs = f_test_imgs / 255.0
f_val_imgs = f_val_imgs / 255.0

In [6]:
db_path='./triple_mnist/'

def splits_flatten_toarray(set):
    set_path=db_path+set
    sub_folders=os.listdir(set_path)

    imgs=[]
    labels=[]

    for folder in sub_folders:
        subfolder_path = os.path.join(set_path, folder)

        for img_file in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, img_file)

            img = Image.open(img_path).convert('L').resize((45,45))
            
            width, height = img.size
            
            split_width = width // 3

            splits = [
                img.crop((i * split_width, 0, (i + 1) * split_width, height))
                for i in range(3)
            ]

            for i, split in enumerate(splits):
                imgs.append(np.array(split))  
                labels.append(int(folder[i]))

    return np.array(imgs), np.array(labels)


t_train_imgs, t_train_labels = splits_flatten_toarray('train')
t_test_imgs, t_test_labels = splits_flatten_toarray('test')
t_val_imgs, t_val_labels = splits_flatten_toarray('val')

t_train_imgs = t_train_imgs / 255.0
t_test_imgs = t_test_imgs / 255.0
t_val_imgs = t_val_imgs / 255.0

In [18]:
db_path='./triple_mnist/'

def splits_flatten_toarray(set):
    set_path=db_path+set
    sub_folders=os.listdir(set_path)

    imgs=[]
    labels=[]

    for folder in sub_folders:
        subfolder_path = os.path.join(set_path, folder)

        for img_file in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, img_file)

            img = Image.open(img_path).convert('L')
            
            width, height = img.size
            
            split_width = width // 3

            splits = [
                img.crop((i * split_width, 0, (i + 1) * split_width, height))
                for i in range(3)
            ]

            for i, split in enumerate(splits):
                imgs.append(np.array(split))  
                labels.append(int(folder[i]))

    return np.array(imgs), np.array(labels)


ft_train_imgs, ft_train_labels = splits_flatten_toarray('train')
ft_test_imgs, ft_test_labels = splits_flatten_toarray('test')
ft_val_imgs, ft_val_labels = splits_flatten_toarray('val')

ft_train_imgs = ft_train_imgs / 255.0
ft_test_imgs = ft_test_imgs / 255.0
ft_val_imgs = ft_val_imgs / 255.0

In [7]:
rs_t_train_labels = t_train_labels.reshape(64000, 3)
rs_f_train_imgs = tf.convert_to_tensor(f_train_imgs)
rs_f_train_imgs = tf.reshape(rs_f_train_imgs, (64000, 84, 84, 1))


print(rs_t_train_labels.shape)
print(rs_f_train_imgs.shape)


(64000, 3)
(64000, 84, 84, 1)


In [75]:
def build_generator():
    # Label embedding
    label_embedding = layers.Embedding(10, 16, name="label_embedding")

    # Label projection
    label_proj = layers.Dense(128 * 6 * 6, name="label_proj")

    # Noise projection
    noise_proj = layers.Dense(256 * 6 * 6, name="noise_proj")

    # Convolutional blocks remain the same
    conv_blocks = tf.keras.Sequential([
        layers.Conv2DTranspose(256, kernel_size=2, strides=7, padding='same', use_bias=False, 
                             input_shape=(384, 6, 6), data_format='channels_first'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', 
                             use_bias=False, data_format='channels_first'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2DTranspose(1, kernel_size=4, strides=1, padding='same', 
                             use_bias=False, data_format='channels_first'),
        layers.Activation('tanh')
    ])

    def generator(noise, labels):
        # Handle dynamic batch size using tf.shape
        batch_size = tf.shape(noise)[0]
        
        # Use tf.slice instead of Python slicing
        labels = tf.slice(labels, [0, 0], [batch_size, -1])
        
        # Embed labels
        emb1 = label_embedding(labels[:, 0])
        emb2 = label_embedding(labels[:, 1])
        emb3 = label_embedding(labels[:, 2])
        
        # Concatenate embeddings
        emb_cat = tf.concat([emb1, emb2, emb3], axis=1)
        
        # Project labels
        label_feat = label_proj(emb_cat)
        label_feat = tf.reshape(label_feat, tf.concat([[batch_size], [128, 6, 6]], axis=0))
        
        # Project noise
        noise_feat = noise_proj(noise)
        noise_feat = tf.reshape(noise_feat, tf.concat([[batch_size], [256, 6, 6]], axis=0))
        
        # Combine features
        combined = tf.concat([noise_feat, label_feat], axis=1)
        
        # Generate output
        output = conv_blocks(combined)
        output = tf.reshape(output, tf.concat([[batch_size], [84, 84, 1]], axis=0))
        
        return output

    return generator

In [77]:
def build_discriminator(dropout_rate=0.3):
    # Label embeddings
    label_embedding_1 = layers.Embedding(10, 16, name="label_embedding_1")
    label_embedding_2 = layers.Embedding(10, 16, name="label_embedding_2")
    label_embedding_3 = layers.Embedding(10, 16, name="label_embedding_3")
    
    # Label projections with controlled output size
    label_proj_blocks = [
        tf.keras.Sequential([
            layers.Dense(64, name=f"label_proj_dense_{i}"),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ], name=f"label_proj_{i}") for i in range(1, 4)
    ]
    
    # Convolutional blocks
    conv_blocks = tf.keras.Sequential([
        layers.Conv2D(32, kernel_size=4, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Dropout(dropout_rate),
        
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Dropout(dropout_rate),
        
        layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Dropout(dropout_rate),
        
        layers.Flatten()
    ], name="conv_blocks")
    
    # Output blocks
    adversarial = tf.keras.Sequential([
        layers.Dense(256),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation="sigmoid")
    ], name="adversarial")
    
    def discriminator(img, labels):
        # Get batch size from input image
        batch_size = tf.shape(img)[0]
        
        # Ensure labels match batch size
        labels = tf.slice(labels, [0, 0], [batch_size, -1])
        
        # Process each label embedding
        emb1 = label_embedding_1(labels[:, 0])
        emb2 = label_embedding_2(labels[:, 1])
        emb3 = label_embedding_3(labels[:, 2])
        
        # Process embeddings through projection blocks
        feat1 = label_proj_blocks[0](emb1)
        feat2 = label_proj_blocks[1](emb2)
        feat3 = label_proj_blocks[2](emb3)
        
        # Combine label features
        label_feat = tf.concat([feat1, feat2, feat3], axis=1)
        
        # Process image through conv blocks
        conv_feat = conv_blocks(img)
        
        # Ensure label_feat has the same batch size as conv_feat
        label_feat = tf.broadcast_to(label_feat, [batch_size, tf.shape(label_feat)[1]])
        
        # Combine features
        combined_features = tf.concat([conv_feat, label_feat], axis=1)
        
        # Generate validity score
        validity = adversarial(combined_features)
        
        return validity
    
    return discriminator

In [10]:
def build_gan(generator, discriminator, latent_dim):
    # Create the input tensors for the GAN
    noise_input = layers.Input(shape=(latent_dim,), name="noise_input")
    label_input = layers.Input(shape=(3,), dtype=tf.int32, name="label_input")
    
    # Generate fake images using the generator
    fake_images = generator(noise_input, label_input)
    
    # Get validity from discriminator
    validity = discriminator(fake_images, label_input)
    
    # Create the GAN model
    gan = tf.keras.Model(inputs=[noise_input, label_input], outputs=validity, name="gan_model")
    
    return gan

In [11]:
def generate_and_visualize(generator, latent_dim):
    noise = np.random.normal(0, 1, (3, latent_dim))  # Generate 3 random noise vectors
    labels = np.random.randint(0, 10, (3, 3))  # Random labels for the generated images
    generated_images = generator.predict([noise, labels])

    # Rescale images to [0, 255] for display
    generated_images = (generated_images + 1) * 127.5
    generated_images = np.clip(generated_images, 0, 255).astype(np.uint8)

    fig, axs = plt.subplots(1, 3, figsize=(10, 5))  # 1 row, 3 columns
    for i in range(3):
        axs[i].imshow(generated_images[i, :, :, 0], cmap='gray')
        axs[i].axis('off')
    
    plt.tight_layout()
    plt.show()


In [12]:
def train_gan(generator, discriminator, gan, train_images, train_labels, latent_dim, epochs, batch_size, display_interval=25):
    num_samples = train_images.shape[0]
    
    for epoch in range(epochs):
        # Generate batch indices
        idx = np.random.randint(0, num_samples, batch_size)
        

        # Get real images and labels
        real_images = tf.gather(train_images, idx)
        real_labels = tf.gather(train_labels, idx)
        
        # Generate noise
        noise = tf.random.normal((batch_size, latent_dim))
        
        # Generate fake images
        fake_images = generator([noise, real_labels], training=True)
        
        # Create labels for training
        real_labels_ = tf.ones((batch_size, 1))
        fake_labels_ = tf.zeros((batch_size, 1))
        
        # Train discriminator
        with tf.GradientTape() as tape:
            # Real images
            real_validity = discriminator([real_images, real_labels], training=True)
            d_loss_real = tf.keras.losses.binary_crossentropy(real_labels_, real_validity)
            
            # Fake images
            fake_validity = discriminator([fake_images, real_labels], training=True)
            d_loss_fake = tf.keras.losses.binary_crossentropy(fake_labels_, fake_validity)
            
            # Total discriminator loss
            d_loss = tf.reduce_mean(d_loss_real + d_loss_fake)
        
        # Apply discriminator gradients
        d_grads = tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator.optimizer.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        
        # Train generator
        noise = tf.random.normal((batch_size, latent_dim))
        with tf.GradientTape() as tape:
            # Generate images
            gen_images = generator([noise, real_labels], training=True)
            # Get discriminator output
            fake_validity = discriminator([gen_images, real_labels], training=True)
            # Calculate generator loss
            g_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(real_labels_, fake_validity)
            )
        
        # Apply generator gradients
        g_grads = tape.gradient(g_loss, generator.trainable_variables)
        generator.optimizer.apply_gradients(zip(g_grads, generator.trainable_variables))
        
        print(f"Epoch {epoch+1}/{epochs} - D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}")

        if (epoch + 1) % display_interval == 0:
            # print(f"Epoch {epoch+1}/{epochs} - D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}")
            generate_and_visualize(generator, latent_dim)


In [78]:
latent_dim = 128  # Size of the noise vector
batch_size = 64
epochs = 250
display_interval = 50

noise = np.random.normal(0, 1, (batch_size, latent_dim))
label_input = rs_t_train_labels
image_input = f_train_imgs

input_noise = tf.keras.Input(shape=(latent_dim,), name="noise_input")
input_labels = tf.keras.Input(shape=(3,), name="label_input")
input_imgs = tf.keras.Input(shape=(84, 84, 1), name="image_input")


# Define the generator and discriminator
generator = build_generator()
discriminator = build_discriminator()

# Build the GAN
gan = build_gan(generator, discriminator, latent_dim)

# Compile the GAN
gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

fake_imgs = generator(input_noise, label_input)
validity = discriminator(input_imgs, label_input)

# Generate the model by calling the function
generator_model = tf.keras.Model(inputs=[input_noise, input_labels], outputs=fake_imgs, name="generator")
discriminator_model = tf.keras.Model(inputs=[input_imgs, input_labels], outputs=[validity], name="discriminator")

generator_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the GAN
train_gan(generator_model, discriminator_model, gan, image_input, label_input, latent_dim, epochs, batch_size, display_interval)

Epoch 1/250 - D Loss: 1.6579, G Loss: 0.7610


KeyboardInterrupt: 